In [25]:
import pandas as pd
import numpy as np
import os
import sys

# take sorted ref genome and filter out for only our desired chromosomes and for euchromatic genes

In [70]:
df = pd.read_csv("sorted_dmel-all-r6.31.gtf", sep='\t', header=None)
df['len_ref'] = (df[4] - df[3]).abs()
df_gene= df.loc[(df[2] == 'gene')].copy()
df_mrna= df.loc[(df[2] == 'mRNA')].copy()
df_gene["gene"] = [x.replace("gene_id","").replace('"',"").strip() for z in df_gene[8].str.split(";").tolist() for x in z if "gene_id" in x]
df_mrna["gene"] = [x.replace("gene_id","").replace('"',"").strip() for z in df_mrna[8].str.split(";").tolist() for x in z if "gene_id" in x]
merged_iso = pd.merge(df_gene, df_mrna, on='gene', how='left')
merged_iso = merged_iso.drop(columns =['1_x','5_x','6_x','7_x','5_y','6_y','7_y','8_y','8_x','1_y','0_y'])

merged_iso.rename(columns={'len_ref_x':'G_len'})
merged_iso.columns =['chrom','feature','G_start','G_end','G_len','G_ID','mRNA','RNA_start','RNA_end','mRNA_count']
merged_iso= merged_iso[['chrom','G_start','G_end','G_len','G_ID','RNA_start','RNA_end','mRNA_count']]

merged_iso= merged_iso.fillna(0)
for col in ['RNA_start','RNA_end','mRNA_count']:
    merged_iso[col] = merged_iso[col].apply(int).apply(str)

merged_iso.loc[merged_iso['mRNA_count'] == '0', 'mRNA_count']= np.nan

merged_iso = merged_iso.groupby(['G_ID']).agg({'chrom':'first','G_start':'first','G_end':'first','G_len':'first',\
	'RNA_start': lambda x: ",".join(x),'RNA_end': lambda x: ",".join(x), 'mRNA_count': lambda x: (x).nunique()}).reset_index()

merged_iso= merged_iso[['chrom','G_start','G_end','G_len','G_ID','RNA_start','RNA_end','mRNA_count']]
merged_iso= merged_iso.fillna(0)
#merged_iso.to_csv('genes_mrna_ref.txt' , index=False,  sep='\t',header=False)

In [71]:
start = [530000,5982495,750000,6754278,1325967]
end = [18870000,24972477,19026900,31614278,21338973]

In [72]:
def trim_euchromatin(df1):

    drop_indexes=[]

    for row in df1.index:
        if df1.at[row,'chrom'] == '2L':
            if df1.at[row,'G_start'] < start[0]:
                drop_indexes.append(row)
            if df1.at[row,'G_end'] > end[0]:
                drop_indexes.append(row)
        if df1.at[row,'chrom'] == '2R':
            if df1.at[row,'G_start'] < start[1]:
                drop_indexes.append(row)
            if df1.at[row,'G_end'] > end[1]:
                drop_indexes.append(row)          
        if df1.at[row,'chrom'] == '3L':
            if df1.at[row,'G_start'] < start[2]:
                drop_indexes.append(row)
            if df1.at[row,'G_end'] > end[2]:
                drop_indexes.append(row)            
        if df1.at[row,'chrom'] == '3R':
            if df1.at[row,'G_start'] < start[3]:
                drop_indexes.append(row)
            if df1.at[row,'G_end'] > end[3]:
                drop_indexes.append(row)        
        if df1.at[row,'chrom'] == 'X':
            if df1.at[row,'G_start'] < start[4]:
                drop_indexes.append(row)
            if df1.at[row,'G_end'] > end[4]:
                drop_indexes.append(row)

    df_euchromatin = df1.drop(drop_indexes)
    return df_euchromatin

In [73]:
df_ref = merged_iso.drop(merged_iso[~merged_iso['chrom'].isin(['X', '3L','3R', '2L','2R'])].index)
df1 = df_ref.copy()

In [74]:
df_euchromatin = trim_euchromatin(df1)

In [75]:
df_euchromatin = df_euchromatin.sort_values(['chrom','G_start'])

In [76]:
def minimum_maximum(minimum, maximum, total):
    if maximum - minimum == 0:
        total = total + 1
    else: 
        total = total + (maximum-minimum + 2)
    return total 

def calc_non_overlap(start,end,minimum,maximum,total):
    if (start > minimum) & (end > maximum):
        minimum = start
        maximum = end
        total = minimum_maximum(minimum, maximum, total)

    elif start <  maximum:
        if  end > maximum:
            minimum = maximum +1
            maximum = end
            total = minimum_maximum(minimum, maximum, total)

    return minimum, maximum, total

In [77]:
df_euchromatin

,chrom,G_start,G_end,G_len,G_ID,RNA_start,RNA_end,mRNA_count
1122,2L,540599,542580,1981,FBgn0010602,"540599,541237,541393,541393","542580,542580,542580,542580",3
4066,2L,542786,544653,1867,FBgn0031260,542786,544653,1
971,2L,544925,552923,7998,FBgn0005660,"544925,547016,547016,547016","552454,549502,552454,552923",4
4067,2L,545129,547096,1967,FBgn0031261,545129,547096,1
13448,2L,553742,560310,6568,FBgn0260933,"553742,553742","559079,560310",2
...,...,...,...,...,...,...,...,...
1886,X,21316052,21317663,1611,FBgn0020369,21316052,21317663,1
12250,X,21317936,21319296,1360,FBgn0064116,"21317936,21317936","21319213,21319296",2
12251,X,21317936,21319296,1360,FBgn0064117,"21317936,21317936","21319296,21319213",2
2955,X,21319791,21324009,4218,FBgn0029067,"21319791,21319791,21319791","21324009,21324009,21324009",1


In [34]:
dict_chrom= {'X':df_euchromatin.loc[df_euchromatin['chrom'] == 'X'], '2L': df_euchromatin.loc[df_euchromatin['chrom'] == '2L'], '2R':df_euchromatin.loc[df_euchromatin['chrom'] == '2R'], '3L':df_euchromatin.loc[df_euchromatin['chrom'] == '3L'], '3R':df_euchromatin.loc[df_euchromatin['chrom'] == '3R']}

dict_totals={}
TOTAL = 0
for key in dict_chrom:
    minimum= 0
    maximum= 0
    total= 0
    for index,row in dict_chrom[key].iterrows():
        minimum, maximum, total = calc_non_overlap(row['G_start'],row['G_end'], minimum, maximum, total )
    dict_totals[key] = total
    print(total)
    TOTAL = TOTAL + total


16157520
13028782
14500457
13087649
18465002


In [8]:
#df_euchromatin['G_len'].sum()

80954737

In [35]:
TOTAL

75239410

In [37]:
75239410/100479888

0.7488006953192464

# Pull in the CNVs and count bases 

In [78]:
df = pd.read_csv('CNVs_genes_categorized.txt', sep='\t',dtype={3: str})
df.head(7)

,CHROM,START,END,CNV_ID,LEN,Flavor,Genomes_present,Num_Alleles,Complex_Event,G_CHROM,...,mRNA_count,Overlap,Num_samples,genic_region,FULL_GENE,+90%_GENE,90-50%_GENE,50-10%_GENE,>10%_GENE,Intergenic
0,2L,454803,455471,0000000001,668,CNV-R,TOM008;B59;I23;T29A;ZH26;A1;A2;A3;A4;A5;A7;AB8...,3,0,2L,...,0,510.0,45,Non_protein,91.234347,True,False,False,False,False
1,2L,481913,490403,0000000002,8490,CNV-Q,RAL426,1,0,2L,...,4,8490.0,1,Protein,13.195524,False,False,True,False,False
2,2L,533286,534017,0000000003,731,CNV-Q,ZH26,1,0,2L,...,4,731.0,1,Protein,1.136152,False,False,False,True,False
3,2L,569560,572845,0000000004,3285,CNV-Q,RAL426,1,2,2L,...,2,3285.0,1,Protein,71.929056,False,True,False,False,False
4,2L,572832,581584,0000000005,8752,CNV-Q,RAL426,1,2,2L,...,2,75.0,1,Protein,1.642216,False,False,False,True,False
5,2L,572832,581584,0000000005,8752,CNV-Q,RAL426,1,2,2L,...,2,1185.0,1,Protein,100.000000,True,False,False,False,False
6,2L,572832,581584,0000000005,8752,CNV-Q,RAL426,1,2,2L,...,1,2063.0,1,Protein,100.000000,True,False,False,False,False


In [40]:
CNV_bases= df.groupby(['CNV_ID']).first()
CNV_bases['Overlap'].sum()

13403151.0

In [54]:
df= df.sort_values(['CHROM','START'])
#CNV_bases= df.groupby(['CNV_ID']).first()

In [55]:
CNV_dict_chrom= {'X':df.loc[df['CHROM'] == 'X'], '2L': df.loc[df['CHROM'] == '2L'], '2R':df.loc[df['CHROM'] == '2R'], '3L':df.loc[df['CHROM'] == '3L'], '3R':df.loc[df['CHROM'] == '3R']}

CNV_dict_totals={}
CNV_TOTAL = 0

for key in CNV_dict_chrom:
    minimum= 0
    maximum= 0
    total= 0
    for index,row in CNV_dict_chrom[key].iterrows():
        minimum, maximum, total = calc_non_overlap(row['START'],row['END'], minimum, maximum, total )
    CNV_dict_totals[key] = total
    print(total)
    CNV_TOTAL = CNV_TOTAL + total

3969913
5570169
5405998
5343091
9473986


In [56]:
#CNV_dict_chrom['3R']['LEN'].sum()
CNV_TOTAL

29763157

# calculate the number of bases involved in genes in the CNVs

In [68]:

df_clean_chroms = df.drop(df[~df['G_CHROM'].isin(['X', '3L','3R', '2L','2R'])].index)
df_clean_chroms = df_clean_chroms.sort_values(['G_CHROM','G_START'])
df_clean_chroms.drop(columns=['mRNA_count','Num_samples','genic_region','FULL_GENE','+90%_GENE','90-50%_GENE','50-10%_GENE','>10%_GENE','Intergenic','mRNA_START',	'mRNA_END'])


,CHROM,START,END,CNV_ID,LEN,Flavor,Genomes_present,Num_Alleles,Complex_Event,G_CHROM,G_START,G_END,G_LEN,G_ID,Overlap
0,2L,454803,455471,0000000001,668,CNV-R,TOM008;B59;I23;T29A;ZH26;A1;A2;A3;A4;A5;A7;AB8...,3,0,2L,454754,455313,559.0,FBgn0262510,510.0
1,2L,481913,490403,0000000002,8490,CNV-Q,RAL426,1,0,2L,476220,540560,64340.0,FBgn0003963,8490.0
2,2L,533286,534017,0000000003,731,CNV-Q,ZH26,1,0,2L,476220,540560,64340.0,FBgn0003963,731.0
3,2L,569560,572845,0000000004,3285,CNV-Q,RAL426,1,2,2L,568340,572907,4567.0,FBgn0031266,3285.0
4,2L,572832,581584,0000000005,8752,CNV-Q,RAL426,1,2,2L,568340,572907,4567.0,FBgn0031266,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11675,X,21763347,21766063,0000007272,2716,CNV-Q,B1;RAL177,2,2,X,21631891,21790731,158840.0,FBgn0267704,2716.0
11676,X,21764415,21765375,0000007273,960,CNV-Q,B59,1,2,X,21631891,21790731,158840.0,FBgn0267704,960.0
11677,X,21765225,21765562,0000007274,337,CNV-Q,B59,1,2,X,21631891,21790731,158840.0,FBgn0267704,337.0
11678,X,21768973,21769629,0000007275,656,CNV-Q,B6,1,1,X,21631891,21790731,158840.0,FBgn0267704,656.0


-158840

In [67]:
CNV_gene_dict_chrom= {'X':df_clean_chroms.loc[df_clean_chroms['G_CHROM'] == 'X'], '2L': df_clean_chroms.loc[df_clean_chroms['G_CHROM'] == '2L'], '2R':df_clean_chroms.loc[df_clean_chroms['G_CHROM'] == '2R'], '3L':df_clean_chroms.loc[df_clean_chroms['G_CHROM'] == '3L'], '3R':df_clean_chroms.loc[df_clean_chroms['G_CHROM'] == '3R']}

CNV_gene_dict_totals={}
CNV_gene_TOTAL = 0

for key in CNV_gene_dict_chrom:
    minimum= 0
    maximum= 0
    total= 0
    for index,row in CNV_gene_dict_chrom[key].iterrows():
        minimum, maximum, total = calc_non_overlap(row['G_START'],row['G_END'], minimum, maximum, total )
    CNV_gene_dict_totals[key] = total
    print(total)
    CNV_gene_TOTAL = CNV_gene_TOTAL + total

10970442
9161845
10055248
10050435
16184125


In [45]:
CNV_gene_TOTAL

51379497

In [19]:
def minimum_maximum(minimum, maximum, total):
    if maximum-minimum == 0:
        total = total + 1
    else: 
        total = total + (maximum-minimum+2)
    return total 

minimum= 0
maximum= 0
total= 0


def calc_non_overlap(df):
    global minimum,maximum,total
    start = df['START']
    end = df['END']

    if (start > minimum) & (end > maximum):
        minimum = start
        maximum = end
        total= minimum_maximum(minimum,maximum,total)

    elif df['START'] <  maximum:
        if df['END'] > maximum:
            minimum = maximum+1
            maximum = df['END']
            total= minimum_maximum(minimum,maximum,total)

    return minimum, maximum, total

In [20]:
df_x = df.loc[df['CHROM'] == 'X']
df_2L = df.loc[df['CHROM'] == '2L']
df_2R = df.loc[df['CHROM'] == '2R']
df_3L = df.loc[df['CHROM'] == '3L']
df_3R = df.loc[df['CHROM'] == '3R']

In [21]:
df_x = df_x.apply(calc_non_overlap, axis=1)
df_2L = df_2L.apply(calc_non_overlap, axis=1)
df_2R = df_2R.apply(calc_non_overlap, axis=1)
df_3L = df_3L.apply(calc_non_overlap, axis=1)
df_3R = df_3R.apply(calc_non_overlap, axis=1)
df_3L 

CNV_ID
0000002709    (25258534, 25260268, 4228192)
0000002710    (25258534, 25260268, 4228192)
0000002711    (25258534, 25260268, 4228192)
0000002712    (25258534, 25260268, 4228192)
0000002713    (25258534, 25260268, 4228192)
                          ...              
0000003977    (25987976, 25988392, 4247718)
0000003978    (26044764, 26045558, 4248514)
0000003979    (26764695, 26766061, 4249882)
0000003980    (27161397, 27161526, 4250013)
0000003981    (27379810, 27379920, 4250125)
Length: 1273, dtype: object

In [67]:
df3L = df_3L[-1][-1]
df3R = df_3R[-1][-1]
df2L = df_2L[-1][-1]
df2R = df_2R[-1][-1]
dfX = df_x[-1][-1]

In [72]:
df3L , df3R , df2L , df2R , dfX

(4250125, 5256765, 3969913, 4228192, 3969913)

In [73]:
df3L + df3R + df2L + df2R + dfX

21674908

In [ ]:
21,674,908

In [ ]:

df_x = df_euchromatin.loc[df_euchromatin['chrom'] == 'X']
df_2L = df_euchromatin.loc[df_euchromatin['chrom'] == '2L']
df_2R = df_euchromatin.loc[df_euchromatin['chrom'] == '2R']
df_3L = df_euchromatin.loc[df_euchromatin['chrom'] == '3L']
df_3R = df_euchromatin.loc[df_euchromatin['chrom'] == '3R']



df_x = df_x.apply(calc_non_overlap, axis=1)
df_2L = df_2L.apply(calc_non_overlap, axis=1)
df_2R = df_2R.apply(calc_non_overlap, axis=1)
df_3L = df_3L.apply(calc_non_overlap, axis=1)
df_3R = df_3R.apply(calc_non_overlap, axis=1)


df3L = df_3L[-1][-1]
df3R = df_3R[-1][-1]
df2L = df_2L[-1][-1]
df2R = df_2R[-1][-1]
dfX = df_x[-1][-1]